In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo
import pandas as pd
from splinter import Browser
import time

In [2]:
# NASA Mars News

# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'

# Retrieve page with the requests module
response = requests.get(url)

# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'lxml')

print(soup.prettify())

<!DOCTYPE html>
<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <!-- Always force latest IE rendering engine or request Chrome Frame -->
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <!-- Responsiveness -->
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <!-- Favicon -->
  <link href="/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
  <link href="/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
  <link href="/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>
  <link href="/manifest.json" rel="manifest"/>
  <link color="#e48b55" href="/safari-pinned-tab.svg" rel="mask-icon"/>
  <meta content="#000000" name="theme-color"/>
  <meta content="authenticity_token" name="csrf-param"/>
  <meta content="8iiYYFGp1ej3NaOsyUgBnxDeAOn92ZY21CGKKCmL0oNbW3HyetPj2+YbtATew9uPC1DLurhfV7mInhvwCcj7fg==" name="csrf-to

In [3]:
# Find div, class slide
result = soup.find('div', class_='slide')

# Scrape the news title
news_title = result.find('div', class_='content_title').text.replace("\n\n", "")

# Scrape the news paragraph text
news_parag = result.find('div', class_='rollover_description_inner').text.replace("\n", "")

print(news_title)
print(news_parag)

NASA Readies Perseverance Mars Rover's Earthly Twin 
Did you know NASA's next Mars rover has a nearly identical sibling on Earth for testing? Even better, it's about to roll for the first time through a replica Martian landscape.


In [4]:
# JPL Mars Space Images - Featured Image

#Set up browser
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

# URL of page to be scraped
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

# Open browser for searching
browser.visit(url)

# Click on full image button
browser.click_link_by_partial_text('FULL IMAGE')

# Time needed to display page
time.sleep(2)

# Click on more info button
browser.click_link_by_partial_text('more info')

# Create BeautifulSoup object; parse with html.parser
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

# Time needed for parsing completition
time.sleep(2)
result = soup.find('figure', class_='lede')

# Relative path
rel_image_url = result.find('a')['href']

# Absolute path
abs_image_url = 'https://www.jpl.nasa.gov' + rel_image_url

# Close browser
browser.quit()

# Print full path
abs_image_url

/Users/cintiaolivamercadomastach/opt/anaconda3/lib/python3.7/site-packages/splinter/driver/webdriver/__init__.py:493: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA22574_hires.jpg'

In [5]:
# Mars Facts

# URL of page to be scraped
url = 'https://space-facts.com/mars/'

# Scrape tabular data
tables = pd.read_html(url)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [6]:
# Name columns 
df = tables[0]
df.columns = ['description', 'value']
df.head()

,description,value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"


In [7]:
# Set the index to the description column
df.set_index('description', inplace=True)
df.head()

,value
description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"


In [8]:
# Convert the data to a HTML table string
html_table = df.to_html()
html_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>value</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\

In [9]:
# Strip unwanted newlines to clean up the table
html_table = html_table.replace('\n', '')
html_table

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>value</th>    </tr>    <tr>      <th>description</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-87 to -5 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

In [10]:
# Mars Hemispheres

#Set up browser
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

# URL of page to be scraped
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

# Open browser for searching
browser.visit(url)

# Create BeautifulSoup object; parse with html.parser
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

# results as an iterable list
results = soup.find_all('div', class_="description")

# List of dictionaries
hemisphere_image_urls = []

# Loop through returned results
for result in results:
    # Error handling
    try:
        # Image title
        title = result.a.h3.text
        
        # Click on image title
        browser.click_link_by_partial_text(title)
        
        # Create a second BeautifulSoup object; parse with html.parser
        html_fullimg = browser.html
        soup_fullimg = BeautifulSoup(html_fullimg, 'html.parser')
        
        # Find full image
        result2 = soup_fullimg.find('div', class_='downloads')
        result_fullimg = result2.find_all('li')
        url_fullimg = result_fullimg[0].a['href']
        
        # Add dictionary to images list
        hemisphere_image_urls.append({'title': title, 'img_url': url_fullimg})
        
        # Back to initial page
        browser.back()
    except AttributeError as e:
        print(e)


# Close browser
browser.quit()

# Print images paths
hemisphere_image_urls

/Users/cintiaolivamercadomastach/opt/anaconda3/lib/python3.7/site-packages/splinter/driver/webdriver/__init__.py:493: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [11]:
# URL of page to be scraped
url = 'https://twitter.com/marswxreport?lang=en'

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'lxml')
print(soup.prettify())

<!DOCTYPE html>
<html dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width,initial-scale=1,maximum-scale=1,user-scalable=0,viewport-fit=cover" name="viewport"/>
  <link href="//abs.twimg.com" rel="preconnect"/>
  <link href="//api.twitter.com" rel="preconnect"/>
  <link href="//pbs.twimg.com" rel="preconnect"/>
  <link href="//t.co" rel="preconnect"/>
  <link href="//video.twimg.com" rel="preconnect"/>
  <link href="//abs.twimg.com" rel="dns-prefetch"/>
  <link href="//api.twitter.com" rel="dns-prefetch"/>
  <link href="//pbs.twimg.com" rel="dns-prefetch"/>
  <link href="//t.co" rel="dns-prefetch"/>
  <link href="//video.twimg.com" rel="dns-prefetch"/>
  <link as="script" crossorigin="anonymous" href="https://abs.twimg.com/responsive-web/client-web-legacy/polyfills.525f28f5.js" nonce="Yzk0ZGRiN2ItYjRlOS00ZWFjLWI1YTItMDRmZTQyMWUwMWVk" rel="preload"/>
  <link as="script" crossorigin="anonymous" href="https://abs.twimg.com/responsive-web/client-web-le

In [12]:
#Set up browser
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

# URL of page to be scraped
url = 'https://twitter.com/marswxreport?lang=en'

# Open browser for searching
browser.visit(url)

# Create BeautifulSoup object; parse with html.parser
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
soup

<html dir="ltr" lang="en" style="font-size: 15px;"><head><meta charset="utf-8"/>
<meta content="width=device-width,initial-scale=1,maximum-scale=1,user-scalable=0,viewport-fit=cover" name="viewport"/>
<link href="//abs.twimg.com" rel="preconnect"/>
<link href="//api.twitter.com" rel="preconnect"/>
<link href="//pbs.twimg.com" rel="preconnect"/>
<link href="//t.co" rel="preconnect"/>
<link href="//video.twimg.com" rel="preconnect"/>
<link href="//abs.twimg.com" rel="dns-prefetch"/>
<link href="//api.twitter.com" rel="dns-prefetch"/>
<link href="//pbs.twimg.com" rel="dns-prefetch"/>
<link href="//t.co" rel="dns-prefetch"/>
<link href="//video.twimg.com" rel="dns-prefetch"/>
<link as="script" crossorigin="anonymous" href="https://abs.twimg.com/responsive-web/client-web/polyfills.931ce165.js" nonce="" rel="preload"/>
<link as="script" crossorigin="anonymous" href="https://abs.twimg.com/responsive-web/client-web/vendors~main.e7e20d65.js" nonce="" rel="preload"/>
<link as="script" crossorigi

In [13]:
# Time needed for parsing completition
#time.sleep(2)
#results = soup.find_all('main', class_='css-11dbjc4n r-1oszu61 r-16y2uox r-1wbh5a2 r-13qz1uu')
#results = soup.find_all('div', class_='css-1dbjc4n r-150rngu r-1oszu61 r-16y2uox r-1wbh5a2 r-13qz1uu')
#results = soup.find_all('div', class_='css-901oao r-jwli3a r-1qd0xha r-a023e6 r-16dba41 r-ad9z0x r-bcqeeo r-bnwqim r-qvutc0')
#results = soup.find_all('span', class_='css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0')
#result = soup.find_all('div', class_="css-901oao r-hkyrab r-1qd0xha r-a023e6 r-16dba41 r-ad9z0x r-bcqeeo r-bnwqim r-qvutc0")
result = soup.find('div', attrs={"class": "tweet", "data-name": "Mars Weather"})
result